# PP

Оставим только реплики менеджера, так как только их надо анализировать. В случае обучения серьезной сети для данной задачи реплики клиента помогли бы лучше выделить необходимые сущности и связи между токенами, но так как пользуемся решениями из коробки (готовые инструменты из библиотек) без возможности обученияя, например, LSTM, пусть будет так). Также приведем все к нижнему регистру, потому что заглавные буквы "смущают" некоторые инструменты (в частности pullenti) иногда распознают в первых словах именованную сущность. 

In [1]:
import spacy
import pandas as pd
import re
from spacy import displacy
from pullenti_wrapper.processor import Processor, PERSON
import stanza
import pymorphy2
from natasha import NamesExtractor

In [2]:
df=pd.read_csv('test_data.csv')
df.head(10)

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага
5,0,5,manager,Угу ну возможно вы рассмотрите и другие вариан...
6,0,6,client,Да мы работаем с компанией которая нам подлива...
7,0,7,client,Как как бы уже до этого момента работаем все у...
8,0,8,manager,Угу а на что вы обращаете внимание при выборе
9,0,9,client,Как бы нет


In [3]:
df=df[df['role']=='manager']
df['text']=df['text'].str.lower()

In [4]:
num_of_dlgs=df['dlg_id'].max()+1

dialogs = [i for i in range(num_of_dlgs)]
for i in range(num_of_dlgs):
    dialogs[i]=df[df['dlg_id']==i]

In [5]:
dialogs[3]

,dlg_id,line_n,role,text
250,3,1,manager,алло дмитрий добрый день
251,3,2,manager,добрый меня максим зовут компания китобизнес у...
253,3,4,manager,да дмитрий вот мне моя коллега анастасия подск...
256,3,7,manager,угу
260,3,11,manager,очень важно я думаю вот например да
265,3,16,manager,да
266,3,17,manager,да абсолютно все то есть это единое такое окно...
268,3,19,manager,так а вот подскажите пожалуйста может быть про...
273,3,24,manager,транспортная компания хорошо а у вас уже вот в...
275,3,26,manager,да но если мы будем прям настраивать вам серым...


# Test NER

In [10]:
test_msg='меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается'

In [15]:
nlp = spacy.load("ru_core_news_lg")
doc = nlp(test_msg)

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

ангелина 11 19 PER


In [16]:
for token in doc:
    token_text = token.text
    token_pos = token.pos_
    token_dep = token.dep_
    token_head = token.head.text
    print(f"{token_text:<12}{token_pos:<10}{token_dep:<10}{token_head:<12}")

Меня        PRON      obj       зовут       
зовут       VERB      ROOT      зовут       
ангелина    PROPN     xcomp     зовут       
компания    NOUN      nsubj     диджитал    
диджитал    VERB      parataxis зовут       
бизнес      NOUN      obj       диджитал    
звоним      VERB      xcomp     зовут       
вам         PRON      iobj      звоним      
по          ADP       case      продления   
поводу      NOUN      fixed     по          
продления   NOUN      obl       звоним      
лицензии    NOUN      nmod      продления   
а           CCONJ     cc        заканчивается
мы          PRON      discourse а           
с           ADP       case      серым       
серым       ADJ       nmod      мы          
у           ADP       case      вас         
вас         PRON      nmod      серым       
скоро       ADV       advmod    заканчивается
срок        NOUN      nsubj     заканчивается
заканчиваетсяVERB      conj      зовут       


In [17]:
from spacy import displacy
displacy.render(doc, style='dep', jupyter=True)

имя определило, название компании - нет, и связи между словами с трудом

In [18]:
def name_recognize_pullenti(text):
    result = ''
    processor = Processor([PERSON])
    ner_result = processor(text)
    if ner_result.matches != []:
        for match in ner_result.matches:
            result += str(match)+ ' '
    if result != '':
        return result
    else: return 0

In [19]:
name_recognize_pullenti('test_msg')

0

In [20]:
stanza.download('ru')
stanza_nlp = stanza.Pipeline(lang='ru', processors='tokenize,ner')

2022-09-03 06:52:35 INFO: Downloading default packages for language: ru (Russian)...
2022-09-03 06:52:38 INFO: File exists: C:\Users\kacha\stanza_resources\ru\default.zip
2022-09-03 06:52:43 INFO: Finished downloading models and saved to C:\Users\kacha\stanza_resources.


2022-09-03 06:52:44 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| ner       | wikiner   |

2022-09-03 06:52:44 INFO: Use device: cpu
2022-09-03 06:52:44 INFO: Loading: tokenize
2022-09-03 06:52:45 INFO: Loading: ner
2022-09-03 06:52:46 INFO: Done loading processors!


In [21]:
def name_recognize_stanza(text):
    result = ''
    global stanza_nlp
    doc = stanza_nlp(text)
    for sent in doc.sentences:
        for ent in sent.ents:
            if ent.type == 'PER':
                result +=  f'entity: {ent.text}\ttype: {ent.type}' + " "
    if result != '':
        return result
    else: return 0

In [22]:
name_recognize_stanza('test_msg')

0

решения библиотек pullenti и stanza не справились с поиском сущностей совсем...

In [23]:
morph = pymorphy2.MorphAnalyzer()

In [24]:
def name_recognize_natasha(text):
    global morph
    extractor = NamesExtractor(morph)
    matches = extractor(text)
    if matches != None:
        result = ''
        for index, match in enumerate(matches):
            if match != None:
                result += str(match.fact)
    if result !='':
        return result
    else:
        return 0

In [25]:
name_recognize_natasha(test_msg)

"Name(first='ангелина', last=None, middle=None)Name(first=None, last='по', middle=None)Name(first=None, last='поводу', middle=None)Name(first=None, last='а', middle=None)Name(first=None, last='мы', middle=None)Name(first=None, last='с', middle=None)Name(first=None, last='серым', middle=None)Name(first=None, last='у', middle=None)Name(first=None, last='скоро', middle=None)"

natasha...

# Parsing

Формулировок приветствия, прощания и представления собеседнику не так уж много, особенно в оффициальных диалогах,
поэтому даннные реплики можно искать по "словарю" (вряд ли кто-то будет упоминать доброту в контексте делового разговора, да и наверняка стандартные формулировки прописаны и все стандартные фразу практически одинаковы).
С названием компании намного труднее, ни одно готовое решение не справилось (не удивительно, учитывая одинаковый регистр, отсутствие пунктуации и качество автоматически созданного текста с зачастую неправильными формами слов), поэтому подпираем алгоритм костылем "наверное, названия всех компаний заканчиваются на 'бизнес'".
Ну и именованные сущности (имя менеджера) определяем с помощью spacy.

In [27]:
privetstviya=['здравствуйте','добрый','доброе','приветствую']
proshaniya=['до свидания','до встречи','всего хорошего','всего доброго', 'прощайте', 'пока' ]

In [31]:
for i in range(num_of_dlgs):
    h_flag = False
    b_flag = False
    i_flag = False
    print(f'\nдиалог номер {i+1}')
    
    for dlg in dialogs[i]['text']:
        for privet in privetstviya:
            if dlg.find(privet) != -1:
                print(f'\nв строке:\n |{dlg}| \nесть приветствие')
                h_flag=True
                break


        
        for poka in proshaniya:
            if dlg.find(poka) != -1:
                print(f'\nв строке:\n |{dlg}|\nесть прощание')  
                b_flag=True
                break


        if (dlg.find('меня') != -1) and (dlg.find('зовут') != -1) or (dlg.find('вас') != -1) and (dlg.find('беспокоит') != -1) :
            doc = nlp(dlg)
            
            i_flag=True
            print(f'\nв строке:\n |{dlg}|\nменеджер представляется, его/ее зовут {doc.ents[0].text}')
            
            lol=re.search(r'компания .*бизнес',dlg)
            if lol is not  None:
                company=dlg[lol.start()+9:lol.end()]
                print(f'он/она из компании {company}')
    print('\nНАРУШЕНИЯ:')
    
    if not h_flag:
        print('менеджер не поздоровался')
    
    if not i_flag:
        print('менеджер не представился')
        
    if not b_flag:
        print('менеджер не попрощался')
    if h_flag and b_flag and i_flag:
        print('отсутствуют\n\n')
    

    
    print('-------------------------------------------------------------------------------------')


диалог номер 1

в строке:
 |алло здравствуйте| 
есть приветствие

в строке:
 |меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается|
менеджер представляется, его/ее зовут ангелина
он/она из компании диджитал бизнес

в строке:
 |всего хорошего до свидания|
есть прощание

НАРУШЕНИЯ:
отсутствуют


-------------------------------------------------------------------------------------

диалог номер 2

в строке:
 |алло здравствуйте| 
есть приветствие

в строке:
 |меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается|
менеджер представляется, его/ее зовут ангелина
он/она из компании диджитал бизнес

в строке:
 |угу да вижу я эту почту хорошо тогда исправлю на эту будем ждать ответа всего хорошего|
есть прощание

в строке:
 |до свидания|
есть прощание

НАРУШЕНИЯ:
отсутствуют


-------------------------------------------------------------------

Благодарю за внимание

Буду рад получить обратную связь и обсудить решение на собеседовании)